In [17]:
import json
import requests as rq
import os
import time
import pandas as pd
import numpy as np
import requests as rq

In [3]:
pd.read_csv("results/idigbio-sample/input/all.tsv", sep="\t").columns

Index(['kingdom', 'phylum', 'family', 'genus', 'specificepithet', 'country',
       'stateprovince', 'county', 'present'],
      dtype='object')

In [15]:
all = pd.read_csv("results/idigbio-sample/input/all.tsv", sep="\t")
all = all[all["kingdom"] != "fungi"]
len(all)

23571

In [16]:
tax_columns = ['kingdom', 'phylum', 'family', 'genus', 'specificepithet']
all[tax_columns].drop_duplicates()

,kingdom,phylum,family,genus,specificepithet
300,plantae,dicotyledon,asteraceae,Acmella,ciliata
301,plantae,dicotyledon,asteraceae,Conyza,bonariensis
302,plantae,dicotyledon,asteraceae,Vernonia,simplex
304,plantae,dicotyledon,asteraceae,Vernonia,megapotamica
305,plantae,dicotyledon,asteraceae,Mikania,congesta
...,...,...,...,...,...
15940,plantae,rhodophyta,rhodomelaceae,Boergeseniella,fruticulosa
15942,plantae,rhodophyta,rhodomelaceae,Rhodomela,lycopodioides
15943,plantae,rhodophyta,rhodomelaceae,Acanthophora,pacifica
15951,plantae,rhodophyta,rhodomelaceae,Amansia,dietrichiana


In [38]:
kpf_counts = rq.get(f"""https://search.idigbio.org/v2/summary/top/records/?top_fields={json.dumps(["kingdom", "phylum", "family"])}""").json()
kpf_counts = { k: v for k, v in kpf_counts["kingdom"].items() if k in ("animalia", "plantae")}
kpf_counts

{'animalia': {'itemCount': 64139807,
  'phylum': {'arthropoda': {'itemCount': 27751117,
    'family': {'formicidae': {'itemCount': 1526878},
     'apidae': {'itemCount': 1375550},
     'carabidae': {'itemCount': 914703},
     'noctuidae': {'itemCount': 812918},
     'halictidae': {'itemCount': 691841},
     'staphylinidae': {'itemCount': 601367},
     'nymphalidae': {'itemCount': 560532},
     'geometridae': {'itemCount': 542437},
     'miridae': {'itemCount': 488580},
     'scarabaeidae': {'itemCount': 465751}}},
   'chordata': {'itemCount': 23423041,
    'family': {'cricetidae': {'itemCount': 1289307},
     'plethodontidae': {'itemCount': 689549},
     'cyprinidae': {'itemCount': 683883},
     'colubridae': {'itemCount': 655612},
     'scincidae': {'itemCount': 533619},
     'hylidae': {'itemCount': 530894},
     'muridae': {'itemCount': 491292},
     'emberizidae': {'itemCount': 436207},
     'phrynosomatidae': {'itemCount': 376115},
     'ranidae': {'itemCount': 364174}}},
   'moll

In [41]:
def kpfc_gen(max_small_groups):
    trouble_makers = {
        # Phyla
        "arthropoda von siebold, 1848",
        "flowering plants",
        "lichens",
        "amoebozoa",
        "fungi incertae sedis",
        "fungi",
        "glomeromycota",

        # Families
        "family unknown"
    }

    for kingdom, k_data in kpf_counts.items():
        for phylum, p_data in k_data["phylum"].items():
            if phylum not in trouble_makers:
                for family, f_data in list(p_data["family"].items())[:max_small_groups]:
                    if family not in trouble_makers:
                        num_records = f_data["itemCount"]
                        yield {"kingdom": kingdom, "phylum": phylum, "family": family, "count": num_records}

# next(kpfc_gen(max_small_groups=1))

In [42]:
for i, x in enumerate(kpfc_gen(max_small_groups=3)):
    print(i, x)

0 {'kingdom': 'animalia', 'phylum': 'arthropoda', 'family': 'formicidae', 'count': 1526878}
1 {'kingdom': 'animalia', 'phylum': 'arthropoda', 'family': 'apidae', 'count': 1375550}
2 {'kingdom': 'animalia', 'phylum': 'arthropoda', 'family': 'carabidae', 'count': 914703}
3 {'kingdom': 'animalia', 'phylum': 'chordata', 'family': 'cricetidae', 'count': 1289307}
4 {'kingdom': 'animalia', 'phylum': 'chordata', 'family': 'plethodontidae', 'count': 689549}
5 {'kingdom': 'animalia', 'phylum': 'chordata', 'family': 'cyprinidae', 'count': 683883}
6 {'kingdom': 'animalia', 'phylum': 'mollusca', 'family': 'unionidae', 'count': 311203}
7 {'kingdom': 'animalia', 'phylum': 'mollusca', 'family': 'muricidae', 'count': 219674}
8 {'kingdom': 'animalia', 'phylum': 'mollusca', 'family': 'veneridae', 'count': 186356}
9 {'kingdom': 'animalia', 'phylum': 'annelida', 'family': 'spionidae', 'count': 41785}
10 {'kingdom': 'animalia', 'phylum': 'annelida', 'family': 'syllidae', 'count': 39538}
11 {'kingdom': 'anim